In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sissopp.sklearn import SISSORegressor
from sissopp.py_interface import read_csv
from sympy import symbols, simplify, expand, sin
from pysr import PySRRegressor
from scipy.special import jn,j0, factorial
import os
import json

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [ ]:
# Dataset
np.random.seed(0)
# Small values of x
x = np.linspace(-1, 1, 500)
#noise = 0.001 * np.random.randn(len(x))
y = jn(0,x)